<a href="https://colab.research.google.com/github/amfakh/eapanicobacoba/blob/master/Scrape_Genius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://medium.com/analytics-vidhya/how-to-scrape-song-lyrics-a-gentle-python-tutorial-5b1d4ab351d2

In [1]:
# Make HTTP requests
import requests

# Scrape data from an HTML document
from bs4 import BeautifulSoup

# I/O
import os

# Search and manipulate strings
import re

In [2]:
GENIUS_API_TOKEN='PXpvxuPl_XyMM02ly-gyw8bd7tyP1avUQLZVsR4ooRmhfbTDOWG8mEHez0DGygjp'

In [3]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs
    
# DEMO
request_song_url('JKT48', 10)

Found 10 songs by JKT48


['https://genius.com/Jkt48-heavy-rotation-lyrics',
 'https://genius.com/Jkt48-fortune-cookie-yang-mencinta-fortune-cookie-in-love-lyrics',
 'https://genius.com/Jkt48-river-lyrics',
 'https://genius.com/Jkt48-pesawat-kertas-365-hari-lyrics',
 'https://genius.com/Jkt48-everyday-kachuusha-lyrics',
 'https://genius.com/Jkt48-dirimu-melody-kimi-wa-melody-lyrics',
 'https://genius.com/Jkt48-only-today-lyrics',
 'https://genius.com/Jkt48-baby-baby-baby-lyrics',
 'https://genius.com/Jkt48-first-rabbit-lyrics',
 'https://genius.com/Jkt48-sebagian-besar-kenangan-omoide-no-hotondo-lyrics']

In [4]:
# For ignoring SSL certificate errors
import urllib.request
import urllib.parse
import urllib.error
import ssl
from urllib.request import Request, urlopen

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [5]:
# # Scrape lyrics from a Genius.com song URL
# def scrape_song_lyrics(url):
#     page = requests.get(url)
#     html = BeautifulSoup(page.text, 'html.parser')
#     lyrics = html.find('div', class_='lyrics').get_text()
#     #remove identifiers like chorus, verse, etc
#     lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
#     #remove empty lines
#     lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
#     return lyrics# DEMO
# print(scrape_song_lyrics('https://genius.com/Jkt48-only-today-lyrics'))

In [6]:
# def write_lyrics_to_file(artist_name, song_count):
#     f = open(artist_name.lower() + '.txt', 'wb')
#     urls = request_song_url(artist_name, song_count)
#     for url in urls:
#         lyrics = scrape_song_lyrics(url)
#         f.write(lyrics.encode("utf8"))
#     f.close()
#     num_lines = sum(1 for line in open('lyrics/' + artist_name.lower() + '.txt', 'rb'))
#     print('Wrote {} lines to file from {} songs'.format(num_lines, song_count))
  
# # DEMO  
# write_lyrics_to_file('JKT48', 5)

In [7]:
song_json = {}
song_json["Lyrics"] = [];

def scrape_from_link(url):
    req = Request(url, headers = { 'User-Agent' : 'Mozilla/5.0' })
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')

    for title in soup.findAll('title'):
        song_json["Title"] = title.text.strip()

    # Extract the release date of the song
    # for span in soup.findAll('span', attrs = {'class': 'metadata_unit-info metadata_unit-info–text_only'}):
    #     song_json["Release date"] = span.text.strip()

    for div in soup.findAll('div', attrs = {'class': 'lyrics'}):
        song_json["Lyrics"].append(div.text.strip().split("\n"));

In [8]:
# scrape_from_link('https://genius.com/Jkt48-only-today-lyrics')

In [11]:
import json

def write_lyrics_to_file(artist_name, song_count):
    f = open(artist_name.lower() + '.json', 'w')
    urls = request_song_url(artist_name, song_count)
    for url in urls:
        print(url)
        scrape_from_link(url)
        print(song_json["Title"])
        # f.write(lyrics.encode("utf8"))
    f.close()

    with open(artist_name.lower() + '.json', 'w') as outfile:
            json.dump(song_json, outfile, indent = 4, ensure_ascii = False)
    # num_lines = sum(1 for line in open(artist_name.lower() + '.txt', 'rb'))
    # print('Wrote {} lines to file from {} songs'.format(num_lines, song_count))
  
# DEMO  
write_lyrics_to_file('Fiersa Besari', 20)

Found 20 songs by Fiersa Besari
https://genius.com/Fiersa-besari-waktu-yang-salah-lyrics
Fiersa Besari – Waktu Yang Salah Lyrics | Genius Lyrics
https://genius.com/Fiersa-besari-celengan-rindu-lyrics
Fiersa Besari – Celengan Rindu Lyrics | Genius Lyrics
https://genius.com/Fiersa-besari-april-lyrics
Fiersa Besari – April Lyrics | Genius Lyrics
https://genius.com/Fiersa-besari-bukan-lagu-valentine-lyrics
Fiersa Besari – Bukan Lagu Valentine Lyrics | Genius Lyrics
https://genius.com/Fiersa-besari-cerita-panjang-dihidup-yang-singkat-lyrics
Fiersa Besari – Cerita Panjang Dihidup yang Singkat Lyrics | Genius Lyrics
https://genius.com/Fiersa-besari-pelukku-untuk-pelikmu-ost-imperfect-karier-cinta-and-timbangan-lyrics
Fiersa Besari – Pelukku Untuk Pelikmu (OST Imperfect: Karier, Cinta, & Timbangan) Lyrics | Genius Lyrics
https://genius.com/Fiersa-besari-sepasang-pendaki-lyrics
Fiersa Besari – Sepasang Pendaki Lyrics | Genius Lyrics
https://genius.com/Fiersa-besari-hidup-kan-baik-baik-saja-lyri